In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from matplotlib import pyplot as plt
import pathlib
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Input, Dropout
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Model
import seaborn as sns
import stumpy


In [ ]:
autoenc = pd.DataFrame(columns=['id', 'anomalies'])

In [ ]:
path = pathlib.Path('/content/phase1')
for path in sorted(path.iterdir()):
  if str(path)=='/content/phase1/.ipynb_checkpoints':
    pass
  else:
    X = np.loadtxt(path)
    df =pd.DataFrame({'value':X})
    filename=str(path).split('\\')[-1]
    print(filename)

    point = int(filename.split('_')[-1].split('.')[0])
    id = int(filename.split('_')[0].split('/')[-1])
    #from univariate to multivariate data
    df['date'] = [i for i in range(len(df))]
    df['velocity'] = df['value'] - df['value'].shift(1)
    df['acc'] = df['velocity'] - df['velocity'].shift(1)
    ws=[10,15,40,50,100,150, 220, 300, 450, 500,600, 700]
    for w in range(ws):
      mps = []
      mps = stumpy.stump(df['value'], w)
      mps_ = [item[0] for item in mps]
      df['mp_'+str(w)] = pd.Series(mps_)

    df.dropna(inplace=True) 
    train, test = df[:point], df[point:]

    scaler = StandardScaler()
    scaler = scaler.fit(train[['value']])

    train['value'] = scaler.transform(train[['value']])
    test['value'] = scaler.transform(test[['value']])

    seq_size = 100  # Number of time steps to look back 
    #Larger sequences (look further back) may improve forecasting.


    def to_sequences(x, y, seq_size=1):
        x_values = []
        y_values = []

        for i in range(len(x)-seq_size):
            #print(i)
            x_values.append(x.iloc[i:(i+seq_size)].values)
            y_values.append(y.iloc[i+seq_size])
            
        return np.array(x_values), np.array(y_values)

    trainX, trainY = to_sequences(train[['value']], train['value'], seq_size)
    testX, testY = to_sequences(test[['value']], test['value'], seq_size)


    model = Sequential()
    model.add(LSTM(128, input_shape=(trainX.shape[1], trainX.shape[2])))
    model.add(Dropout(rate=0.2))

    model.add(RepeatVector(trainX.shape[1]))

    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(rate=0.2))
    model.add(TimeDistributed(Dense(trainX.shape[2])))
    model.compile(optimizer='adam', loss='mae')
    model.summary()

    # fit model
    history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

    plt.plot(history.history['loss'], label='Training loss')
    plt.plot(history.history['val_loss'], label='Validation loss')
    plt.legend()

    #model.evaluate(testX, testY)

    trainPredict = model.predict(trainX)
    trainMAE = np.mean(np.abs(trainPredict - trainX), axis=1)
    plt.hist(trainMAE, bins=30)
    max_trainMAE = 0.3 

    testPredict = model.predict(testX)
    testMAE = np.mean(np.abs(testPredict - testX), axis=1)
    plt.hist(testMAE, bins=30)

    #Capture all details in a DataFrame for easy plotting
    anomaly_df = pd.DataFrame(test[seq_size:])
    anomaly_df['testMAE'] = testMAE
    anomaly_df['max_trainMAE'] = max_trainMAE
    anomaly_df['anomaly'] = anomaly_df['testMAE'] > anomaly_df['max_trainMAE']
    anomaly_df['value'] = test[seq_size:]['value']

    anomalies = anomaly_df.loc[anomaly_df['anomaly'] == True]
    autoenc= autoenc.append({'id':id, 'anomalies' : list(anomalies['date'])[0]},ignore_index=True)

# New Section